# EvoGFuzz++

In [1]:
import string
import math
from fuzzingbook.Grammars import Grammar

GRAMMAR: Grammar = {
    "<start>": ["<arith_expr>"],
    "<arith_expr>": ["<function>(<number>)"],
    "<function>": ["sqrt", "sin", "cos", "tan"],
    "<number>": ["<maybe_minus><onenine><maybe_digits><maybe_frac>"],
    "<maybe_minus>": ["", "-"],
    "<onenine>": [str(num) for num in range(1, 10)],
    "<digit>": list(string.digits),
    "<maybe_digits>": ["", "<digits>"],
    "<digits>": ["<digit>", "<digit><digits>"],
    "<maybe_frac>": ["", ".<digits>"]
}

INITIAL_INPUTS = ['cos(10)', 'sqrt(28367)', 'tan(-12)', 'sqrt(3)']

Let's parse the initial input files an observe if they are valid according to the grammar

In [2]:
from fuzzingbook.Parser import EarleyParser, tree_to_string, DerivationTree

p = EarleyParser(GRAMMAR)

for inp in INITIAL_INPUTS:
    for tree in p.parse(inp):
        assert tree_to_string(tree) == inp, f"{tree} != {inp}"

In [3]:
def arith_eval(inp: DerivationTree) -> float:
    return eval(str(inp), {"sqrt": math.sqrt, "sin": math.sin, "cos": math.cos, "tan": math.tan})

# Returns true if a Error is found
def prop(inp: DerivationTree) -> bool:
    try:
        arith_eval(inp)
        return False
    except ValueError:
        return True


In [4]:
print([(x, prop(x)) for x in INITIAL_INPUTS])

[('cos(10)', False), ('sqrt(28367)', False), ('tan(-12)', False), ('sqrt(3)', False)]


In [5]:
from importlib import reload  # Not needed in Python 2
import logging
reload(logging)
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s:  %(message)s"
)

In [6]:
from typing import Tuple, Set

def fitness_function(data: Set[Tuple[DerivationTree, bool]]) -> Set[Tuple[DerivationTree, bool, float]]:
    fitness_data = set()
    for inp in data:
        fitness = get_fitness(inp)
        fitness_data.add((inp[0], inp[1], fitness))

    return fitness_data

def get_fitness(inp: Tuple[DerivationTree, bool]) -> int:
    if inp[1]:
        return 1
    else:
        return 0

In [7]:
from evogfuzz.evogfuzz_class import EvoGFuzz
epp = EvoGFuzz(GRAMMAR,
               prop,
               INITIAL_INPUTS,
               fitness_function=fitness_function,
               working_dir=None
               )

In [8]:
epp.execute()

2023-01-07 14:20:47,285:  Executing EvoGFuzz
2023-01-07 14:20:47,287:  Executing input files
2023-01-07 14:20:47,289:  Learned probabilistic grammar:
2023-01-07 14:20:47,290:  <start>                                 [('<arith_expr>', {'prob': None})]
2023-01-07 14:20:47,291:  <arith_expr>                            [('<function>(<number>)', {'prob': None})]
2023-01-07 14:20:47,292:  <function>                              [('sqrt', {'prob': None}), ('sin', {'prob': None}), ('cos', {'prob': None}), ('tan', {'prob': None})]
2023-01-07 14:20:47,292:  <number>                                [('<maybe_minus><onenine><maybe_digits><maybe_frac>', {'prob': None})]
2023-01-07 14:20:47,293:  <maybe_minus>                           [('', {'prob': None}), ('-', {'prob': None})]
2023-01-07 14:20:47,294:  <onenine>                               [('1', {'prob': None}), ('2', {'prob': None}), ('3', {'prob': None}), ('4', {'prob': None}), ('5', {'prob': None}), ('6', {'prob': None}), ('7', {'prob': Non